In [1]:
import bigframes.pandas as bpd

In [ ]:
# Set BigQuery project and location
bpd.options.bigquery.project = "your-project-id"
bpd.options.bigquery.location = "us-central1"

# Prepare The Environment

In [ ]:
import gspread
import json
import pandas as pd
from datetime import datetime
from oauth2client.service_account import ServiceAccountCredentials

In [ ]:
from IPython.core.magic import register_cell_magic
from IPython import get_ipython


@register_cell_magic
def skip(line, cell):
    return


@register_cell_magic
def skip_if(line, cell):
    if eval(line):
        return
    get_ipython().ex(cell)

# Authenticate and create the client

In [ ]:
# Authenticate and create the client
scope = ["https://spreadsheets.google.com/feeds",
         "https://www.googleapis.com/auth/drive"]
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    'drive-api.json', scope)
googleClient = gspread.authorize(credentials)

# Prepare The Data

In [ ]:
# Prepare The Data
# Open the Google Sheet by ID
sheet_id = "1kQENu6sumzEQX60fjQtgmXvwPGlUfaNRgW7v_TWFUXo"
sheet = googleClient.open_by_key(sheet_id)

# Select the first worksheet
worksheet = sheet.get_worksheet(0)

# Get all records from the worksheet
legacy_charges = worksheet.get_all_records(numericise_ignore=['all'])

# Set null to empty mid_label on legacy_charges
for record in legacy_charges:
    record['mid_label'] = None if record['mid_label'] == '' else record['mid_label']
    record['installment_count'] = None if record['installment_count'] == '' else int(
        record['installment_count'])

# Convert the records to a BigQuery DataFrame
legacy_charges_df = bpd.DataFrame(legacy_charges)
legacy_charges_df['installment_count'] = legacy_charges_df['installment_count'].astype(
    'Int64')

In [7]:
# Open the Google Sheet by ID
sheet_id = "1_8sm8QciAU3T8oDlNS1Pfj-GQlmlJBrAi1TYdnnMlkw"
sheet = googleClient.open_by_key(sheet_id)

# Select the first worksheet
worksheet = sheet.get_worksheet(0)

# Get all records from the worksheet
merchant_send_mid_label = worksheet.get_all_records()

# Convert the records to a BigQuery DataFrame
merchant_send_mid_label_df = bpd.DataFrame(merchant_send_mid_label)

In [8]:
# Open the Google Sheet by ID
sheet_id = "1orVBlPP77HTt9d8x-lC1Oo5xrPp0r1FgVUQ-43DYqYc"
sheet = googleClient.open_by_key(sheet_id)

# Select the first worksheet
worksheet = sheet.get_worksheet(0)

# Get all records from the worksheet
merchant_excluded = worksheet.get_all_records()

# Convert the records to a BigQuery DataFrame
merchant_excluded_df = bpd.DataFrame(merchant_excluded)

In [9]:
legacy_charges_df.head()

,business_id,card_brand,card_type,issuing_bank_name,currency,installment_count,client_type,merchant_id,mid_label
0,bu-963,JCB,DEBIT,MANDIRI,IDR,3,WEB,merch-def,promo-mid
1,bu-651,VISA,DEBIT,BNI,IDR,6,API,merch-ghi,promo-mid
2,bu-972,JCB,DEBIT,MANDIRI,IDR,<NA>,WEB,merch-jkl,promo-mid
3,bu-318,MASTERCARD,DEBIT,CIMB,IDR,3,WEB,merch-mno,promo-mid
4,bu-456,VISA,CREDIT,OTHER,IDR,3,WEB,merch-ghi,promo-mid


# Monitoring Script

In [ ]:
legacy_charges_df.to_gbq('temp.legacy_charges', if_exists='replace')
merchant_send_mid_label_df.to_gbq(
    'temp.merchant_send_mid_label', if_exists='replace')
merchant_excluded_df.to_gbq('temp.merchant_excluded', if_exists='replace')

'temp.merchant_excluded'

In [12]:
sql = """
SELECT
    lc.*
FROM
    `temp.legacy_charges` lc
LEFT JOIN
    `temp.merchant_send_mid_label` msml ON lc.business_id = msml.business_id
LEFT JOIN
    `temp.merchant_excluded` me ON lc.business_id = me.business_id
WHERE
    msml.business_id IS NULL AND me.business_id IS NULL
"""
sql_join_df = bpd.read_gbq(sql)

In [13]:
sql_join_df.head()

,business_id,card_brand,card_type,issuing_bank_name,currency,installment_count,client_type,merchant_id,mid_label
0,bu-154,AMEX,CREDIT,BCA,IDR,3,MOBILE,merch-def,promo-mid
1,bu-129,AMEX,CREDIT,BCA,IDR,12,MOBILE,merch-ghi,special-offer
2,bu-134,AMEX,CREDIT,BCA,IDR,3,WEB,merch-jkl,promo-mid
3,bu-636,AMEX,DEBIT,BNI,IDR,3,MOBILE,merch-jkl,<NA>
4,bu-104,AMEX,DEBIT,BNI,IDR,6,API,merch-def,special-offer
